## Loading Packages & Initialization

In [1]:
rm(list=ls())

library(data.table)
library(tidyverse)
library(rJava)
library(RNetLogo)
library(lhs)

options(warn = 0)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.tab

In [2]:
folder.path = "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/"
source(paste0(folder.path,"ThesisSetupCode.r"))

Is_Headless <- 1
nl.model <- "info_cascade_update_TDP_JPF_2020" #"Segregation_Dummy"

nl.path <- "/Users/ecemnaz.yildiz/Documents/NetLogo 6.0.4/Java"
folder.path = "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/"

model.path <- paste0(folder.path, nl.model, ".nlogo")

if (Is_Headless == 0) {
    NLStart(nl.path, gui = TRUE, nl.jarname = "netlogo-6.0.4.jar")
    NLLoadModel(model.path)
} else {
    NLStart(nl.path, gui = FALSE, nl.jarname = "netlogo-6.0.4.jar", nl.obj = nl.model)
    NLLoadModel(model.path, nl.obj = nl.model)
}


Attaching package: ‘plotly’

The following object is masked from ‘package:igraph’:

    groups

The following object is masked from ‘package:ggplot2’:

    last_plot

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout

Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin

Warning message:
“package ‘factoextra’ was built under R version 3.6.3”Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa
Loading required package: gridExtra
Warning message:
“package ‘gridExtra’ was built under R version 3.6.3”
Attaching package: ‘gridExtra’

The following object is maske

In [3]:
model.type = "info_cascade_update" ##ifelse(nl.model == "Segregation", "basic", "dummy")
# the path of data folder

training_set_size = 500 #75 #500
training_set_seed = 9 #8
training_set_date = "2022-02-13"

test_set_size = 215 #30 #215
test_set_seed = 8
test_set_date = "2022-02-12"

data.path = paste0(folder.path,"Data_",training_set_size,"_Seed",training_set_seed,"/")

# the path for outputs to be record
output.folder = paste0("outputs_V3_RFE_mtrymultip2_",model.type,"_",Sys.Date(),"_",format(Sys.time(), "%H.%M"),"_",training_set_size,"_",training_set_seed)
dir.create(file.path(folder.path, output.folder), showWarnings = FALSE)

outputs.path = paste0(folder.path,output.folder,"/")

# Read Me File to keep info about the output folder
ReadMe = paste0(outputs.path,"ReadMe_",model.type,".txt")

## Model Parameters & Functions

### Set model parameters

In [4]:
#### Model Parameters #### Set model parameters Number of replications for each
#### instance
nofrep = 30 #############################

# order feature names according to their definition order in run_model

    feature_names = c(
    "max_links",
    "evidence",
    "sc-bel-prop",
    "prop-likelihood",
    "n_init_believers",
    "prior-mean",
    "prior-sd",
    "expertise_influence") 
    feature_ranges = data.table(  feature   = feature_names
                                , min_range = c(2, 0, 0, 0, 0, 0, 0, 0)
                                , max_range = c(500, 100, 5, 1, 100, 1, 1, 1)
                               )

# 
output_name = c("cl-prop-same")

# Number of input parameters of the agent-based model
nofparams = length(feature_names)

# set RF parameters
ntree = 300
#mtry = 2
mtry.multiplier = 2 # when 1, it is default, when 2, it is at most twice of defaults 
nperm = 5

### Set user parameters

In [5]:
#### User parameters ####
error_type = "RMSE"  # MAPE, BIAS

# choose the uncertainty measure
selection_metric <- "coefvar"  #, 'range' 
sample.type = paste0("AdFe_",selection_metric)

elimination.type = "NRFE" # or "RFE"

# Number of iterations
iteration_budget = 11
metarep = c(1:10)

# Number of instances
unlabeled_ins = 30
test_ins = 215 #30 #215 ##c(100,400)
train_ins_oneshot = 500 #75 #500
train_ins_Ad = 500 #75 #500 ##50

# Set selection parameters
selected_ins = 5  #nofinstancesWillbeSelected in each step

# Set elimination parameter
p = 0.2 # elimination proportion
# h = 1
oob_allowance = 0.01

seed.focus = 9 ##c(1,2,3,4,5,6,7,8,9,20)

## !!!
unlabeled.type = "refresh and ElimInducedSampling"

# Decide on strategy:
elimination_start_iter = 5

log_entry()

## Test Set

In [6]:
#### Test Sets ####
test_set = data.table()
for( t in test_set_size){
    test_set.name= paste0(data.path,"test_set_",model.type,"_",t,"_seed",test_set_seed,"_",test_set_date,".csv")
    test_set_Sub <- fread(test_set.name)  
    
    test_set = rbind(test_set, data.table(size = t, test_set_Sub))
    
    #assign(paste0("test_set_",t),test_set)
}

In [ ]:
paste0(data.path,"test_set_",model.type,"_",t,"_seed",test_set_seed,"_",test_set_date,".csv")

In [7]:
test_set

size,max_links,evidence,sc-bel-prop,prop-likelihood,n_init_believers,prior-mean,prior-sd,expertise_influence,output
215,189.62460,91.883719,3.4096457,0.65294353,21.931290,0.81369670,0.38252934,0.546690542,48.22549
215,472.26499,28.560495,1.1881089,0.92273792,80.046443,0.98328880,0.70990444,0.304951987,43.75605
215,302.24971,26.433859,2.8147611,0.13756179,66.341151,0.22553450,0.32349952,0.884159073,47.58689
215,51.32996,54.971811,1.6453674,0.95689327,63.857945,0.77174551,0.09226774,0.560788175,44.70953
215,31.44839,50.460513,0.2971720,0.52391192,5.029048,0.25692528,0.32124878,0.930769383,69.01723
215,125.92689,37.370317,4.0521681,0.82448262,53.921749,0.81556397,0.26011453,0.068728022,66.62976
215,95.03907,18.922892,0.2424934,0.15100679,45.736426,0.98324116,0.18267836,0.568441012,49.07328
215,144.51384,2.125038,2.5202274,0.07575313,83.823701,0.50919590,0.74961552,0.576853669,42.83901
215,192.34215,55.098017,1.3591612,0.28615150,50.847169,0.58258702,0.98952283,0.526410916,45.50651
215,217.46080,72.178856,4.8820877,0.76007216,34.005433,0.44495482,0.99197711,0.927555483,46.84836


## Adaptive Training Set

In [7]:
adaptive_initial_data = upload_training_set(model.type,training_set_seed,training_set_size,training_set_date)

In [9]:
unique(adaptive_initial_data$seed)

[1] 9

### Adaptive & Feature Elimination Train & Test Metamodel

In [8]:
sample.type = paste0("AdFe_",selection_metric)
sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

for(i in seed.focus){ print(paste0("seed : ",i,"  Adaptive Sampling with Feature Selection section start time : ",Sys.time()))    
    for (r in metarep){ print(paste0("seed : ", i,"   rep : ", r, "  Adaptive Sampling with Feature Selection section start time : ", Sys.time()))
        set.seed(i + r)
            
        training_set_Ad = copy(adaptive_initial_data[seed == i, .SD, .SDcols = -c("seed")])
        train_candidates_table = data.table()
        
        columns_left = feature_names # reset at the beginning of each iteration
        total_numof_eliminated_vars <- 0 # reset at the beginning of each iteration
        eliminated_columns = c()
    
        iteration_history = data.table("seed" = integer(),"rep" = integer(),"iter_no" = integer()
                              ,"IsFeatureEliminated" = logical(), "IsDataSelected" = logical()
                              ,"NumOfEliminated" = integer(), "RankedUpd" = logical())
        iter = 1
        while(iter <= iteration_budget){   
            print(iter)
            run_log_entry()
    
            trainx = training_set_Ad[,.SD, .SDcols = columns_left]
            trainy = training_set_Ad$output
        
            run_step_log_entry("Model Training Start.")
            
            set.seed(9) ### KONTROL
            
            # Train the model
            model_Sub <- randomForest( x = trainx, y =  trainy,importance = TRUE
                                      ,ntree = ntree, nperm = nperm
                                      ,mtry = mtry_default(columns_left) * mtry.multiplier)
                model_Sub.name = paste0("model_",sample.type,"_", iter, "_seed_", i, "_rep_",r)
                model_Sub.path = paste0(outputs.path,models.folder, paste0(model_Sub.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
                saveRDS(model_Sub, model_Sub.path)
        
            iteration_history= rbind(iteration_history,data.table(i,r,iter,0,0,0,0), use.names = FALSE)
            # update VIM or not
            if (elimination.type == "RFE" | (elimination.type == "NRFE" & (length(columns_left) == length(feature_names)))){
                ranked_features = get_variable_importance(model_Sub)
                iteration_history[iter]$RankedUpd= 1 
                
            }     
       
            # write errors 
            obb_err = obb_error_func(model_Sub)     
            fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
                   ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
        
            write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
            write_importance.rf(i,r,iter,model_Sub,sample.type)#last one=sample_type
        
            if(iter != iteration_budget){ # below efforts are unnecessary when the budget is reached. 
                
                run_step_log_entry("Sample Selection Start.")
         
                ### SAMPLE SELECTION ###    
                #select samples first but not to add to the training set until eliminated_features are specified.
                # select new data candidates before elimination
                ## sample selection from unlabeled data select candidates
                unlabeled_set <- refresh_sample_pool(i + r + iter, columns_left)
                train_candidates = sample_selection(selected_ins, unlabeled_set, model_Sub,selection_metric)
                
                run_step_log_entry("ABM Run Start.")
                
                # run ABM to find outputs of train candidates
                print(paste0("ABM train_candidate run start time : ",Sys.time()))
                train_candidates = run_ABM(nofrep, selected_ins, train_candidates)
                
                run_step_log_entry("ABM Run End.")
                
                print(paste0("ABM train_candidate run end time : ",Sys.time()))
                
                fwrite(data.table(train_candidates, "iter" = iter, "seed" = i, "rep" = r)
                       ,paste0(outputs.path,sample.folder,model.type,"_train_candidates_table_",sample.type,".csv"),append = TRUE )      

                ### SAMPLE SELECTION ENDS ###
                
                ### FEATURE ELIMINATION ###
                if(elimination_start_iter <= iter & length(columns_left) >= 2){ #######ilk deneylerde eşitlik yoktu.
                    check_elim = TRUE 
                    apply_elim = FALSE
                    # 
                ### FEATURE ELIMINATION PART I ###
                #decide how many features will be eliminated
                    elim_check_iter = 1
                    h = floor(length(columns_left) * (p^elim_check_iter))
                    while(check_elim){
                        
                        set.seed(9) ### KONTROL
                        
                        run_step_log_entry("Feature Selection Start.")
    
                        # Assume as if feature(s) will be eliminated
                        feature_elimination_result = feature_elimination(h, columns_left, ranked_features)
                        planned_columns_left = feature_elimination_result[[1]]
                        
                        run_step_log_entry("New Random Forest Model Generation.")
                        
                        model_Sub_afterElim <- randomForest(  x = training_set_Ad[,.SD, .SDcols = planned_columns_left]
                                                             ,y =  training_set_Ad$output
                                                             ,importance = TRUE, nperm = nperm
                                                             ,ntree = ntree
                                                            , mtry = mtry_default(planned_columns_left) * mtry.multiplier)        
                            model_Sub_afterElim.name = paste0("model_afterElim_",sample.type,"_", iter, "_seed_", i, "_rep_",r,"_h_",h)
                            model_Sub_afterElim.path = paste0(outputs.path,models.folder, paste0(model_Sub_afterElim.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
                            saveRDS(model_Sub_afterElim, model_Sub_afterElim.path)

                        run_step_log_entry("New Random Forest Model OOB Calculation.")

                        new_oob = obb_error_func(model_Sub_afterElim)
                        
                        fwrite(data.table(iter,new_oob_error = new_oob,oob_error = obb_err,seed = i,rep = r)
                               ,paste0(outputs.path,sample.folder,model.type,"_","new_oob_error_",sample.type,".csv"),append = TRUE)
        
                        if(new_oob < (obb_err + obb_err * oob_allowance)){ 

                            run_step_log_entry("New Random Forest Model Selected.")

                            check_elim = FALSE 
                            apply_elim = TRUE                            

                        } else {
                            
                            run_step_log_entry("New Random Forest Model is not Selected.")

                            elim_check_iter = elim_check_iter + 1
                            h_upd = floor(length(columns_left) * (p^elim_check_iter)) 
                            
                            if(h_upd == h){ # if h does not change
                                check_elim = FALSE    
                            }
                            
                            h = copy(h_upd)
                        }
                     }             
               ### FEATURE SELECTION PART II ###
               # really eliminate 
                    if(apply_elim){
                        
                        run_step_log_entry("Feature Elimination Applied.")
                        
                        # update iteration_history
                        iteration_history[iter]$IsFeatureEliminated= 1
                        iteration_history[iter]$NumOfEliminated= length(columns_left) - length(planned_columns_left)
                
                        columns_left = planned_columns_left
                        eliminated_columns =  feature_elimination_result[[4]]

                        run_step_log_entry("Eliminated Columns Recorded.")
                    
                    }         
               }
              ### FEATURE SELECTION ENDS ###
            
              # add labeled candidates to the train data
              training_set_Ad = rbind(training_set_Ad, train_candidates[, -c("idx")],use.names = TRUE)
              # update iteration_history
              iteration_history[iter]$IsDataSelected= 1

                run_step_log_entry("Labeled Data Added to Training Set.")
  
            }
            fwrite(iteration_history[iter],paste0(outputs.path,sample.folder,model.type,"_iteration_history_",sample.type,".csv"),append = TRUE )       

            run_step_log_entry(paste0("Iteration history Updated. Iteration ", iter, " Ends."))

            iter = iter + 1

        }
        
        run_step_log_entry("Final Train Data File Recorded.")
    
        fwrite(data.table(training_set_Ad, "seed" = i,"rep" = r),paste0(outputs.path,sample.folder,model.type,"_FinalTrainData_",sample.type,".csv") ,append = TRUE)

        run_step_log_entry("Eliminated Columns File Recorded.")

        fwrite(data.table("seed" = i,"rep" = r, "elim_cols" =  eliminated_columns),paste0(outputs.path,sample.folder,model.type,"_EliminatedColumns_",sample.type,".csv") ,append = TRUE)

        run_step_log_entry(paste0("seed : ",i,"   rep : ", r,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))       
                       
    ##    print(paste0("seed : ",i,"   rep : ", r,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))
    }
    
    run_step_log_entry(paste0("seed : ",i,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))                 
                     
##    print(paste0("seed : ",i,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))
    #rm(training_set_Ad,predictedLabels_table,train_candidates_table)      
}

[1] "seed : 9  Adaptive Sampling with Feature Selection section start time : 2022-03-01 23:08:13"
[1] "seed : 9   rep : 1  Adaptive Sampling with Feature Selection section start time : 2022-03-01 23:08:13"
[1] 1
[1] "ABM train_candidate run start time : 2022-03-01 23:08:14"
[1] "ABM train_candidate run end time : 2022-03-01 23:10:47"
[1] 2
[1] "ABM train_candidate run start time : 2022-03-01 23:10:48"
[1] "ABM train_candidate run end time : 2022-03-01 23:13:50"
[1] 3
[1] "ABM train_candidate run start time : 2022-03-01 23:13:51"
[1] "ABM train_candidate run end time : 2022-03-01 23:16:28"
[1] 4
[1] "ABM train_candidate run start time : 2022-03-01 23:16:29"
[1] "ABM train_candidate run end time : 2022-03-01 23:19:59"
[1] 5
[1] "ABM train_candidate run start time : 2022-03-01 23:20:00"
[1] "ABM train_candidate run end time : 2022-03-01 23:23:31"
[1] 6
[1] "ABM train_candidate run start time : 2022-03-01 23:23:33"
[1] "ABM train_candidate run end time : 2022-03-01 23:26:53"
[1] 7
[1] "ABM

## Quit NL

In [9]:
NLQuit(nl.obj = nl.model)